In [ ]:
%run /Utils/Functions/core

In [ ]:
import requests
import concurrent.futures
import time
import sys
import os
import re
from time import sleep, mktime
from uuid import uuid4
import json
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta, timezone

In [ ]:
## Cria as variáveis de LOG
format_log = '%Y-%m-%d %H:%M:%S'
dtInicio = datetime.today() - timedelta(hours=3)
dtInicio_format = dtInicio.strftime(format_log)
tipo_log = 'API'
camada = 'landing'
emissor = '<org>'
atividade = '<activity_desc>'
origem = 'RestService'
destino = 'AzureBlobFS'
execUrl = ' '

try:
    infos = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson()) # captura as informações do job que executa o notebook
    orgId = infos['tags']['orgId']
    runId = infos['tags']['multitaskParentRunId']
    jobId = infos['tags']['jobId']
    if orgId == '2960871991268730': # Monta a URL caso seja o ID do ambiente de DEV
        execUrl = f'https://adb-{orgId}.10.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
    else: # Monta a URL caso seja o ID do ambiente de PROD
        execUrl = f'https://adb-{orgId}.15.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
except:
    print('Campo de URL não pode ser identificado!')

In [ ]:
%sql
use catalog prod;

In [ ]:
dbutils.widgets.text("dt_ingestao", "")

dt_ingestao = getArgument("dt_ingestao").upper().strip()

location_landing = spark.sql("show external locations").select("url").where("name = 'landing-area'").collect()[0][0]
location_flat_file = spark.sql("show external locations").select("url").where("name = 'flatfile-area'").collect()[0][0]

print(location_landing)
print(location_flat_file)

In [ ]:
# Formata o dt_ingestao
format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
dtIngestao = datetime.now() if dt_ingestao == "" else datetime.strptime(dt_ingestao, format_timestamp)

# Pega o horário atual e tira 3 horas para converter para BRT (UTC -3)
# Tira 5 minutos por conta de um limite da API
dt_fim = dtIngestao - timedelta(hours=3) - timedelta(minutes=5)
dt_inicio = dt_fim - timedelta(days=1)

# Força as horas, minutos e segundos em 0
dt_inicio = datetime(dt_inicio.year, dt_inicio.month, dt_inicio.day, 0, 0, 0)

# Separa as datas no formato UNIX
dt_fim_unix = int(mktime(dt_fim.timetuple()))
dt_inicio_unix = int(mktime(dt_inicio.timetuple()))

# dt_inicio_unix = 1546300800

# Transforma as datas em STRING utilizando um formato de timestamp
dt_fim = dt_fim.strftime(format_timestamp)
dt_inicio = dt_inicio.strftime(format_timestamp)

print(f"dt_inicio: {dt_inicio} | unix: {dt_inicio_unix}")
print(f"dt_fim   : {dt_fim} | unix: {dt_fim_unix}")

In [ ]:
domain = "googleads"
domain_api = "googleapis"
baseUrl_access_token = f"https://www.{domain_api}.com"
baseUrl_api = f"https://{domain}.{domain_api}.com"
endpoint_api = f"/v15/customers/@CUSTOMER_ID/googleAds:searchStream"
endpoint_token = "/oauth2/v4/token"

threads = 10

print(baseUrl_access_token)
print(baseUrl_api)

In [ ]:
data_access_token = {
    "client_id": "",
    "client_secret": "",
    "refresh_token": "",
    "grant_type": "refresh_token",
}

In [ ]:
def get_Access_Token(baseUrl, endpoint_token,data):
    try:
        url = baseUrl + endpoint_token

        response_access_token = requests.post(url, data=data)

        response_data = response_access_token.json()

        access_token = response_data["access_token"]
        
        return access_token
    
    except Exception as error:
        print(f'ERROR: {error}')

In [ ]:
access_token = get_Access_Token(baseUrl_access_token, endpoint_token, data_access_token)

print(access_token)

In [ ]:
headers = {
        "Authorization": f"Bearer {access_token}",
        "developer-token": "",
        "Content-Type": "application/json"
        }

threads = 10

In [ ]:
# Pega apenas cargas que possuem valor preenchido no campo "ds_Url" para não trazer tabelas que são geradas a partir de outro evento na landing.
df_param = fn_ConsultaJdbc("""
    SELECT pca.*
    FROM ctl.ADF_Parametro_Carga_API pca
    WHERE pca.fl_ativo = 1
    and nm_Sistema = 'google_ads'
    and pca.id_Carga_Sistema = 97
    and ds_Url is not null
""")

data_param = df_param.collect()

display(df_param)

In [ ]:
def get_tables(param,baseUrl_api,endpoint_api,headers,dt_ingestao, location_landing):
    end_point_page = None
    try:
        format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
        dtIngestao = str(dt_ingestao)
        
        vlUltimoIncremento = str(param.vl_Ultimo_Incremento.split(' ')[0])

        if param.vl_Schedule_Carga != 0:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
        else:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()

        
        url = baseUrl_api + endpoint_api.replace('@CUSTOMER_ID', '5296577110')

        query = {"query": param.ds_Url.replace('@VL_ULTIMO_INCREMENTO', vlUltimoIncremento).replace('@DT_INGESTAO', dtIngestao.split(' ')[0])}

        response = requests.post(url, headers=headers, data=json.dumps(query))

        if response.status_code == 200:

            response_data = response.json()

            result_data = []
            
            for dados in response_data:
                for resultado in dados['results']:
                    result_data.append(resultado)
            
            fn_SaveJson(result_data, dir_landing, str(param.ds_Nome_Arquivo_Landing).lower())
            fn_AtualizaUltimoIncremento_API(param.id_Parametro_Carga_API, dtIngestao)
            
        else:
            print(f'Falha na requisição da tabela {param.nm_Item_Origem}')
    

        # LOG SUCESSO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "execUrl": execUrl}

        fn_LogSucceeded(parametros, dt_ingestao.strftime(format_log))

        return f"Carga da tabela {param.ds_Nome_Arquivo_Landing} finalizada com sucesso."

    except Exception as error:
        # print(error)
        # LOG ERRO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '

        if hasattr(error, 'code'): # captura o código do erro, caso possua o atributo 'code'
            error_code = error.code
        else:
            error_code = 'NULL'
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "cd_erro": error_code, "erro": str(error), "execUrl": execUrl}

        fn_LogFailed(parametros, dt_ingestao.strftime(format_log))

In [ ]:
# Cria uma lista para armazenar todas as tarefas
tasks = []

# Cria uma instância do ThreadPoolExecutor com threads definidas (max_workers)
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    # Percorre todas as pastas do diretório de origem
    for row in data_param:
        # Executa a função fn_StreamFromFolder_csv em uma thread do ThreadPoolExecutor
        task = executor.submit(get_tables(row,baseUrl_api,endpoint_api,headers,dtIngestao,location_landing))
                               
        # Adiciona a tarefa à lista de tarefas
        tasks.append(task)

# Aguarda a conclusão de todas as tarefas
_ = concurrent.futures.wait(tasks, return_when='ALL_COMPLETED')

In [ ]:
for task in tasks:
    try:
        print(task.result(),'\n')
    except Exception as error:
        print(error)
        pass